In [1]:
# Original: https://stackoverflow.com/a/45466355/673294 user: jdehesa

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.
    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

In [3]:
import tensorflow as tf

inp = tf.placeholder(name="inp", dtype=tf.float32, shape=(1, 1))
w = tf.Variable(tf.zeros([1, 1], tf.float32), dtype=tf.float32, name="w")

y = tf.matmul(w, inp)

out = tf.identity(y, name="out")

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    
    # After Init var, change the value to 2
    assignment = w.assign([[2]])
    sess.run(assignment)
    
    output = sess.run(out, feed_dict={inp: [[1]]})
    print(output)
    
    frozen_graph = freeze_session(sess, output_names=[out.op.name])

    tflite_model = tf.contrib.lite.toco_convert(frozen_graph, [inp], [out])
    open("app/src/main/assets/mat_mul.tflite", "wb").write(tflite_model)

[[2.]]
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
